---
# Exercício Titanic - kNN, Regressão Logística
---
**Machine Learning em Projetos (Erick Muzart e Fernando melo)**   
Tópicos:
- One-hot Encoder get-dummies
- Knn
- Regressão logística
- Matriz de confusão

<br>
------------------------------------------------
   
Usando dados do Titanic, descritos em: https://www.openml.org/d/40945

## Carregar bibliotecas

In [1]:
# Importar bibliotecas pandas, plot_confusion_matrix, train_test_split, matplotlib, metrics
import pandas as pd

## Análise exploratória dos dados

In [2]:
# Ler dados do Titanic a partir do site Openml.org
df = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl', na_values='?')
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [3]:
from autoML import AutoML
automl = AutoML(df, y_colname='survived')
automl.getResults()

AttributeError: 'float' object has no attribute 'lower'

In [1]:
# Mostrar as dimensões e as primerias linhas do dataset


In [2]:
# Mostra total de valores nulos por variável independente (features)


In [ ]:
# Fazer um corte no dataset com as colunas:'age', 'pclass', 'sex', 'embarked', 'survived' e eliminar dados nulos.


In [3]:
# Mostrar novas dimensões do dataset após o corte


In [4]:
# Verifica balanceamento das classses (value_counts())


In [5]:
#mostra gráfico de barras do total de amostras por classe


## Pré-processamento

In [6]:
# Atribuir valores para X e y

# Mostrar as dimensões de X e y


### Encoding variáveis (features) categoricas :
Se as variáveis não possuem uma ordem específica, geralmente a melhor opção é one hot-encoding (dummy encoding)

In [7]:
# Cria X com as features codificadas com ".get_dummies" do Pandas.

# Mostra o resultado da codificação one-hot encoding


In [8]:
# Dividir dados em treino e teste (70/30)


## Treinamento dos modelos

### Knn - K vizinhos mais próximos (K Nearest neighbors)

In [ ]:
# Importar o modelo knn Classifier


In [ ]:
# Instanciar modelo de regressão logística


In [9]:
# Treinar o modelo com os dados de treinamento


In [10]:
# Verificar a acurácia do modelo (.score)


In [11]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


### Regressão Logística

In [ ]:
# Instanciar modelo de regressão logística


In [12]:
# Treinar o modelo com os dados de treinamento


In [13]:
# Verificar a acurácia do modelo (.score)


In [14]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


## Avaliação da qualidade do melhor modelo

In [ ]:
# Importar a biblioteca (plot_confusion_matrix)


In [15]:
# Mostrar a Matriz de Confusão do melhor modelo treinado


## Próximos passos
- Fazer normalização dos dados (Standard Scalar)   
- Treinar outros modelos:   
  - Árvore de decisão (Decision Tree Classifier)   
  - Floresta aleatória (Random Forest Classifier)

In [ ]:
# Árvore de decisão
clf = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
clf.score(X_test, y_test)

0.8121019108280255

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_class = RandomForestClassifier(n_estimators=120, max_depth=6, random_state=100).fit(X_train, y_train)
rf_class.score(X_test, y_test)

0.802547770700637